In [9]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
pd.set_option('display.max_rows', None)
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor


In [10]:
output_path = "../data/processed/merged_df.csv"
merged_df = pd.read_csv(output_path)
merged_df = merged_df.rename(columns={'total_year': 'HDD_total_year'})

# GDP per capita
merged_df['GDP_per_capita'] = merged_df['GDP_2022'] / merged_df['Population_2023']

# Delete Taiwan (temporal)
merged_df = merged_df[merged_df['Country'] != 'Taiwan']

np.random.seed(42)
merged_df['is_train'] = np.random.choice([1, 0], size=len(merged_df), p=[0.8, 0.2])

In [11]:


def calculate_metrics(y_true, y_pred):
    return {
        "MSE": mean_squared_error(y_true, y_pred),
        "R²": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAPE": np.mean(np.abs((y_true - y_pred) / y_true)) * 100,
        "WAPE": np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100
    }

def train_and_evaluate_model(X_train, X_test, y_train, y_test, model_type='lr'):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    if model_type == 'lr':
        model = LinearRegression()
    elif model_type == 'knn':
        model = KNeighborsRegressor(n_neighbors=5)
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    return calculate_metrics(y_test, y_pred)

def add_metadata(metrics, region, target, strategy, features_used, model_type):
    model_name = 'Linear Regression' if model_type == 'lr' else 'KNN'
    metrics.update({
        'Region': region,
        'Target': target,
        'Strategy': strategy,
        'Model': model_name,
        'Features Used': ', '.join(features_used)
    })
    return metrics

def reorder_columns(df):
    column_order = [
        'Region', 'Target', 'Strategy', 'Model', 'Features Used', 'Features Abbreviated',
        'MSE', 'R²', 'MAE', 'RMSE', 'MAPE', 'WAPE'
    ]
    existing_columns = df.columns.tolist()
    final_columns = [col for col in column_order if col in existing_columns]
    return df[final_columns]

def within_domain(df, features, target_columns, model_type):
    results = []
    for region in df['Region Grouped'].unique():
        region_data = df[df['Region Grouped'] == region].copy()
        mask = region_data['is_train'] == 1
        
        for target in target_columns:
            X_train = region_data[features][mask]
            X_test = region_data[features][~mask]
            y_train = region_data[target][mask]
            y_test = region_data[target][~mask]
            
            metrics = train_and_evaluate_model(X_train, X_test, y_train, y_test, model_type)
            results.append(add_metadata(metrics, region, target, 'Within-Domain', features, model_type))
    return reorder_columns(pd.DataFrame(results).round(2))

def cross_domain(df, features, target_columns, model_type):
    results = []
    regions = df['Region Grouped'].unique()
    
    for test_region in regions:
        train_data = df[df['Region Grouped'] != test_region]
        test_data = df[df['Region Grouped'] == test_region]
        
        for target in target_columns:
            X_train = train_data[features]
            X_test = test_data[features]
            y_train = train_data[target]
            y_test = test_data[target]
            
            metrics = train_and_evaluate_model(X_train, X_test, y_train, y_test, model_type)
            results.append(add_metadata(metrics, test_region, target, 'Cross-Domain', features, model_type))
    return reorder_columns(pd.DataFrame(results).round(2))

def all_domain(df, features, target_columns, model_type):
    results = []
    regions = df['Region Grouped'].unique()
    
    train_mask = df['is_train'] == 1
    X_train_all = df[features][train_mask]
    
    for region in regions:
        test_mask = (df['Region Grouped'] == region) & (df['is_train'] == 0)
        test_data = df[test_mask]
        
        for target in target_columns:
            y_train = df[target][train_mask]
            y_test = test_data[target]
            X_test = test_data[features]
            
            metrics = train_and_evaluate_model(X_train_all, X_test, y_train, y_test, model_type)
            results.append(add_metadata(metrics, region, target, 'All-Domain', features, model_type))
    return reorder_columns(pd.DataFrame(results).round(2))

def add_abbreviated_feature_column(df, features, feature_abbreviations):
    abbreviated_features = [feature_abbreviations.get(feature, feature) for feature in features]
    df['Features Abbreviated'] = ' | '.join(abbreviated_features)
    return df

def append_total_row(df):
    total_row_df = pd.DataFrame()
    
    for target in df['Target'].unique():
        target_df = df[df['Target'] == target]
        metrics_avg = target_df[['MSE', 'R²', 'MAE', 'RMSE', 'MAPE', 'WAPE']].mean()
        
        total_row = {
            'Region': 'Total',
            'Target': target,
            'Strategy': df['Strategy'].iloc[0],
            'Model': df['Model'].iloc[0],
            'Features Used': df['Features Used'].iloc[0],
            'Features Abbreviated': df.get('Features Abbreviated', [''])[0],
            'MSE': metrics_avg['MSE'],
            'R²': metrics_avg['R²'],
            'MAE': metrics_avg['MAE'],
            'RMSE': metrics_avg['RMSE'],
            'MAPE': metrics_avg['MAPE'],
            'WAPE': metrics_avg['WAPE']
        }
        
        total_row_df = pd.concat([total_row_df, pd.DataFrame([total_row])], ignore_index=True)
    
    return reorder_columns(pd.concat([df, total_row_df], ignore_index=True))

def filter_and_concatenate(*dfs):
    filtered_dfs = [df[df['Region'] == 'Total'] for df in dfs]
    concatenated_df = pd.concat(filtered_dfs, ignore_index=True)
    return concatenated_df

def run_all_strategies(df, features, target_columns, model_type):
    within_domain_results = within_domain(df, features, target_columns, model_type)
    cross_domain_results = cross_domain(df, features, target_columns, model_type)
    all_domain_results = all_domain(df, features, target_columns, model_type)
    return within_domain_results, cross_domain_results, all_domain_results



In [12]:
features = [
    "HDD_total_year", 
    "GDP_per_capita", 
    "Urbanization_Rate_2022",
    "latitude",
    "longitude",  
    "Paris_Agreement",
    "Subnational HDI", 
    "Educational index", 
    "Income index"
]

feature_abbreviations = {
    "HDD_total_year": "HDD",
    "GDP_per_capita": "GDP",
    "Urbanization_Rate_2022": "URB",
    "latitude": "Lat",
    "longitude": "Long",
    "Paris_Agreement": "Paris",
    "Subnational HDI": "HDI", 
    "Educational index": "EDU" , 
    "Income index": "Income"
}

target_columns = [
    "Residential EUI (kWh/m2/year)",
    "Non-residential EUI (kWh/m2/year)"
]

In [13]:
# Linear Regression
within_domain_results_lr, cross_domain_results_lr, all_domain_results_lr = run_all_strategies(
    df=merged_df,
    features=features,
    target_columns=target_columns,
    model_type='lr'
)

# KNN
within_domain_results_knn, cross_domain_results_knn, all_domain_results_knn = run_all_strategies(
    df=merged_df,
    features=features,
    target_columns=target_columns,
    model_type='knn'
)

# Linear Regression
within_domain_results_lr = add_abbreviated_feature_column(within_domain_results_lr, features, feature_abbreviations)
cross_domain_results_lr = add_abbreviated_feature_column(cross_domain_results_lr, features, feature_abbreviations)
all_domain_results_lr = add_abbreviated_feature_column(all_domain_results_lr, features, feature_abbreviations)

within_domain_results_lr = append_total_row(within_domain_results_lr)
cross_domain_results_lr = append_total_row(cross_domain_results_lr)
all_domain_results_lr = append_total_row(all_domain_results_lr)

# KNN
within_domain_results_knn = add_abbreviated_feature_column(within_domain_results_knn, features, feature_abbreviations)
cross_domain_results_knn = add_abbreviated_feature_column(cross_domain_results_knn, features, feature_abbreviations)
all_domain_results_knn = add_abbreviated_feature_column(all_domain_results_knn, features, feature_abbreviations)

within_domain_results_knn = append_total_row(within_domain_results_knn)
cross_domain_results_knn = append_total_row(cross_domain_results_knn)
all_domain_results_knn = append_total_row(all_domain_results_knn)

# Concatenate results
concatenated_results_lr = filter_and_concatenate(
    within_domain_results_lr,
    cross_domain_results_lr,
    all_domain_results_lr
)

concatenated_results_knn = filter_and_concatenate(
    within_domain_results_knn,
    cross_domain_results_knn,
    all_domain_results_knn
)

In [14]:
concatenated_results_lr

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Total,Residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2829.180,-0.798,31.582,46.284,23.988,22.414
1,Total,Non-residential EUI (kWh/m2/year),Within-Domain,Linear Regression,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2630.322,-3.456,28.974,41.872,19.406,16.376
2,Total,Residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2613.266,-1.918,40.806,48.450,40.592,34.984
3,Total,Non-residential EUI (kWh/m2/year),Cross-Domain,Linear Regression,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,4238.268,-12.850,50.058,60.408,31.890,31.102
4,Total,Residential EUI (kWh/m2/year),All-Domain,Linear Regression,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2751.310,-0.786,34.390,46.124,26.714,25.358
5,Total,Non-residential EUI (kWh/m2/year),All-Domain,Linear Regression,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2931.040,-10.062,38.778,45.420,23.592,21.328


In [15]:
concatenated_results_knn

,Region,Target,Strategy,Model,Features Used,Features Abbreviated,MSE,R²,MAE,RMSE,MAPE,WAPE
0,Total,Residential EUI (kWh/m2/year),Within-Domain,KNN,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2579.676,0.158,27.374,40.848,18.658,18.778
1,Total,Non-residential EUI (kWh/m2/year),Within-Domain,KNN,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,1791.388,0.128,17.222,28.374,12.014,9.464
2,Total,Residential EUI (kWh/m2/year),Cross-Domain,KNN,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2519.698,-1.692,36.686,47.394,31.234,30.360
3,Total,Non-residential EUI (kWh/m2/year),Cross-Domain,KNN,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,3506.438,-10.726,36.128,52.374,21.234,20.876
4,Total,Residential EUI (kWh/m2/year),All-Domain,KNN,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,2617.868,0.112,26.864,41.352,17.596,18.520
5,Total,Non-residential EUI (kWh/m2/year),All-Domain,KNN,"HDD_total_year, GDP_per_capita, Urbanization_R...",HDD | GDP | URB | Lat | Long | Paris | HDI | E...,1360.262,-0.016,16.854,26.902,11.520,9.362


In [16]:
# SAVE RESULTS 
results_dir = '../results/'
os.makedirs(results_dir, exist_ok=True)

date_str = datetime.now().strftime("%Y%m%d_%H%M")

# Linear Regression
within_domain_results_lr.to_csv(
    os.path.join(results_dir, f'results_{date_str}_within_domain_lr.csv'), 
    index=False
)
cross_domain_results_lr.to_csv(
    os.path.join(results_dir, f'results_{date_str}_cross_domain_lr.csv'), 
    index=False
)
all_domain_results_lr.to_csv(
    os.path.join(results_dir, f'results_{date_str}_all_domain_lr.csv'), 
    index=False
)
concatenated_results_lr.to_csv(  
    os.path.join(results_dir, f'results_{date_str}_total_lr.csv'), 
    index=False
)

# KNN
within_domain_results_knn.to_csv(
    os.path.join(results_dir, f'results_{date_str}_within_domain_knn.csv'), 
    index=False
)
cross_domain_results_knn.to_csv(
    os.path.join(results_dir, f'results_{date_str}_cross_domain_knn.csv'), 
    index=False
)
all_domain_results_knn.to_csv(
    os.path.join(results_dir, f'results_{date_str}_all_domain_knn.csv'), 
    index=False
)
concatenated_results_knn.to_csv(  
    os.path.join(results_dir, f'results_{date_str}_total_knn.csv'), 
    index=False
)
